# 실습 7: 웹 애플리케이션 연동

## 목표
- Flask API 서버 생성 (WebSocket URL 지원)
- Frontend 설정 (env.js)
- 전체 시스템 테스트

## 7-1. 설정 확인

In [ ]:
from workshop_config import REGION, WINDY_API_KEY, AGENT_RUNTIME_ARN

print(f"🌍 Region: {REGION}")
print(f"🌤️ Windy API Key: {'설정됨' if WINDY_API_KEY else '⚠️ 미설정'}")
print(f"🤖 Agent ARN: {AGENT_RUNTIME_ARN if AGENT_RUNTIME_ARN else '⚠️ 미설정 - lab6 먼저 실행'}")

---
## 7-2. Flask 서버 생성 (WebSocket URL 지원)

In [ ]:
from workshop_config import REGION, AGENT_RUNTIME_ARN

main_content = f'''"""Flask API Server for Fire Response Agent"""

import boto3
import json
import re
from flask import Flask, request, Response, stream_with_context, jsonify
from flask_cors import CORS
from botocore.config import Config

app = Flask(__name__)
CORS(app)

config = Config(read_timeout=3000, connect_timeout=600, retries={{"max_attempts": 0}})
client = boto3.client("bedrock-agentcore", region_name="{REGION}", config=config)

AGENT_RUNTIME_ARN = "{AGENT_RUNTIME_ARN}"


@app.route("/ws-url", methods=["POST"])
def get_websocket_url():
    """Pre-signed WebSocket URL 생성"""
    data = request.json
    session_id = data.get("session_id", "default-session")
    
    try:
        from bedrock_agentcore.runtime import AgentCoreRuntimeClient
        runtime_client = AgentCoreRuntimeClient(region="{REGION}")
        
        presigned_url = runtime_client.generate_presigned_url(
            runtime_arn=AGENT_RUNTIME_ARN,
            session_id=session_id,
            expires=300
        )
        return jsonify({{"url": presigned_url}})
    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({{"error": str(e)}}), 500


@app.route("/analyze", methods=["POST"])
def analyze():
    data = request.json
    prompt = data.get("prompt", "")
    actor_id = data.get("actor_id", "default-user")
    session_id = data.get("session_id", "default-session")

    payload = json.dumps({{
        "prompt": prompt,
        "actor_id": actor_id,
        "session_id": session_id
    }}).encode()

    response = client.invoke_agent_runtime(
        agentRuntimeArn=AGENT_RUNTIME_ARN,
        runtimeSessionId=f"workshop-{{session_id}}-{{abs(hash(actor_id)) % 10000000000}}",
        payload=payload
    )

    def generate():
        if "text/event-stream" in response.get("contentType", ""):
            for line in response["response"].iter_lines(chunk_size=10):
                if line:
                    line = line.decode("utf-8")
                    if line.startswith("data: "):
                        match = re.search(r"'data':\\s*'([^']*)'" , line)
                        if match:
                            yield match.group(1)

    return Response(stream_with_context(generate()), content_type="text/plain; charset=utf-8")


if __name__ == "__main__":
    print("🚀 Flask 서버 시작: http://localhost:8082")
    app.run(host="0.0.0.0", port=8082, debug=True)
'''

with open('../agent/main.py', 'w') as f:
    f.write(main_content)

print("✅ agent/main.py 생성 완료")

---
## 7-3. env.js 생성

In [ ]:
from workshop_config import WINDY_API_KEY

print("SageMaker Studio URL을 확인하세요.")
print("예: https://xxxxx.studio.us-west-2.sagemaker.aws")

STUDIO_DOMAIN = input("\nStudio 도메인 입력: ").strip()

agent_api_url = f'{STUDIO_DOMAIN}/jupyterlab/default/proxy/8082/ws-url'

env_js_content = f'''const ENV = {{
    USER_ID: 'workshop-user',
    SESSION_ID: 'workshop-session',
    WINDY_API_KEY: '{WINDY_API_KEY}',
    WINDY_API_URL: 'https://api.windy.com/api/point-forecast/v2',
    AGENT_API_URL: '{agent_api_url}',
}};
'''

with open('../env.js', 'w') as f:
    f.write(env_js_content)

print("\n✅ env.js 생성 완료")
print(f"📝 AGENT_API_URL: {agent_api_url}")

---
## 7-4. 서버 실행 스크립트 생성

In [ ]:
%%writefile ../start_server.sh
#!/bin/bash
echo "🚀 Flask 서버 시작..."
cd agent
python main.py

In [ ]:
%%writefile ../start_http_server.sh
#!/bin/bash
echo "🌐 HTTP 서버 시작 (포트 8000)..."
python3 -m http.server 8000

In [ ]:
!chmod +x ../start_server.sh ../start_http_server.sh
print("✅ 실행 스크립트 생성 완료")

---
## 7-5. 서버 실행 방법

### 터미널 1: Flask 서버 (Backend)
```bash
./start_server.sh
```

### 터미널 2: HTTP 서버 (Frontend)
```bash
./start_http_server.sh
```

### 접근 주소 (SageMaker Studio)
```
Frontend: https://{studio-domain}/jupyterlab/default/proxy/8000/
Backend:  https://{studio-domain}/jupyterlab/default/proxy/8082/
```

---
## 7-6. 테스트 쿼리

브라우저에서 Frontend 접속 후 다음 쿼리로 테스트:

```
서울특별시 서초구 방배중앙로 06681에서 화재가 발생했습니다. 
가까운 소방서를 찾아주세요.
```

### 예상 결과
- ✅ 가까운 소방서 5곳 표시
- ✅ 지도에 마커 표시
- ✅ 기상 정보 (풍속, 풍향, 온도)
- ✅ CCTV 검색 (천안 지역)

---
## 7-7. 전체 시스템 요약

In [ ]:
from workshop_config import MEMORY_ID, AGENT_RUNTIME_ARN, VECTOR_BUCKET_NAME, CCTV_BUCKET_NAME

print("="*60)
print("🎉 워크샵 완료!")
print("="*60)
print("\n📋 생성된 리소스:")
print(f"   - Memory ID: {MEMORY_ID}")
print(f"   - 소방서 Vector Bucket: {VECTOR_BUCKET_NAME}")
print(f"   - CCTV Vector Bucket: {CCTV_BUCKET_NAME}")
print(f"   - Agent ARN: {AGENT_RUNTIME_ARN}")
print("\n📁 생성된 파일:")
print("   - workshop_config.py (공유 설정)")
print("   - agent/main.py (Flask 서버 + WebSocket URL)")
print("   - agent/deploy/runtime/* (Agent Runtime)")
print("   - env.js (Frontend 설정)")
print("\n🚀 실행 방법:")
print("   1. 터미널1: ./start_server.sh")
print("   2. 터미널2: ./start_http_server.sh")
print("   3. 브라우저에서 Frontend 접속")
print("="*60)

---
## ✅ 실습 7 완료!

### 🎉 전체 워크샵 완료!

### 구축한 시스템
1. **Memory 시스템** - Short-term & Long-term Memory
2. **Vector Database** - S3 Vectors 기반 소방서 + CCTV 검색
3. **Agent Runtime** - Bedrock Agent Core (WebSocket 지원)
4. **Tools** - 소방서 검색, CCTV 검색, 기상 정보, 위키피디아
5. **Web Application** - Flask + Frontend (WebSocket URL 지원)